In [ ]:

'''
This Notebook is for training an ML model with GPU resources on Digital Research Alliance Canada (DRAC)
The data set consists of 4000 h5-format files.  Each file contains 24 hours of data.  

Within the h5 files, there is a data group called '/data' with many multi channel variables.  
There is also a set of annotations that indicate the classication of the data when anomalies are present.  
Generally the class-imbalance for anomalous data is very large.  ( ~1 day with anomalous data per year)

Components:
1) Custom data loader that loads the data
- the data is for an ADCP with 3 beams.  Each beam collects data for beam velocity, beam backscatter, and beam correlation.  
- The beams can be thought of as independant data sets, with 3 input channels, so each beam can be fed through the model independantly.  
- The "anomalous" data may only occur for 6 hours within a 24 hour period, 
- We want a model to identify specifically which data is anomalous (not just binary yes/no for the entire 24 hour period)

2) Define a model architecture / type
 - This has been designed to be modular, to experiment with differnet options for the architecture 

3) Loss function 
- is weighted to account for class-imbalance.  
- binary dice loss can be used for single class (true/false)
- graduated dice loss can be used for multi-class
- also combined with BCE loss for stability in training

'''

In [ ]:

'''
To set up the environment:

conda create -n adcp_anomaly_env python=3.10
conda activate adcp_anomaly_env
pip install -r requirements.txt

OR

python -m venv adcp_anomaly_env
adcp_anomaly_env\Scripts\activate
pip install -r requirements.txt

'''

In [ ]:
#Some changes:
# I updated dataset_loader so that MINOR A and MINOR B classes are not included
# By this I mean, they are not excluded from training, they just have class = 0 (same as normal data)
# I chose to do this because it was super-ceding proper anomalous data, and really skewing the model training
# => This change lead to SIGNIFICANT improvement in classification

#Session Settings on JupyterHub
#Memory: 15000
#Cores: 4
#GPUs: 1

#Kernel: SSAMBA Kernel on Scratch

#if DRAC is being buggy, try from ssh:
# $ salloc --account=def-kmoran --time=02:00:00 --mem=16G --cpus-per-task=4 --gres=gpu:h100:1


In [5]:
# Cell 1: Imports & Setup

import os
import torch
from torch.utils.data import DataLoader, random_split

# Add repo root to Python path - Needed to import from src folder
import sys
from pathlib import Path
repo_root = Path().resolve().parent  # notebooks/ → ADCP-CNN-QAQC
sys.path.append(str(repo_root))

from src.dataset_loader import ADCPDataset  # your custom dataset
from src.resnet_temporal import ResNetTemporalClassifier # CNNClassifier  # your model
# from model import TemporalCNN # CNNClassifier  # your model
from src.utils import seed_everything, get_class_weights, combined_loss, train_model

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

seed_num = 42
seed_everything(seed_num)


#IMPORTANT NOTE: ALSO NEED TO ADJUST THIS IN "utils.py" TO MATCH, in "def train_model()"

USE_WANDB = False # This is a vestigal implementation from my local computer. I did not have success with WANDB on DRAC resources
#USE_WANDB = True  # TODO: Set to True to enable logging - Keep false while in development/debugging
if USE_WANDB:
    import wandb
    
DEBUG_MODE = False

In [2]:
# Cell 2: Initialize Configuration
#53842d9970aafed0ab407079e403fe03469dcb33

from types import SimpleNamespace

USE_WANDB = False # not working on rorqual, so setting to false

if USE_WANDB:
    wandb.init(project="adcp-anomaly-detection", #dir="/scratch/ML_ADCP/wandb_runs", 
            config={
                "model": "ResNetTemporalClassifier",
                "epochs": 100,
                "batch_size": 16,
                "lr": 1e-3,
                "loss_alpha": 0.5,
                "optimizer": "Adam"
            })
    config = wandb.config
else:
    config = SimpleNamespace(**{
        "model": "ResNetTemporalClassifier",
        "epochs": 100,
        "batch_size": 16, #2,
        "lr": 1e-3,
        "loss_alpha": 0.5,
        "optimizer": "Adam"})

In [17]:
#Example: 
#Tools to reload packages if debugging:
import importlib
import dataset_loader
importlib.reload(dataset_loader)

from src.dataset_loader import ADCPDataset

In [3]:
# Cell 3: Load Dataset

from sklearn.model_selection import train_test_split

#Get a list of files from the directory:
data_folder = "/lustre10/scratch/slonimer/BACAX_24hr_h5/" # Rorqual
#data_folder = "/scratch/slonimer/ML_ADCP/BACAX_24hr_h5/" # Fir / Cedar
#data_folder= r"F:\Documents\Projects\ML\ADCP_ML\h5_24h_files\\" # Local Windows VM

file_list = os.listdir(data_folder)
h5_files = {k for k in file_list if os.path.splitext(k)[1] == ".h5"}
#Files are inherently NOT in order in python! So need to do this:
h5_files = sorted(h5_files)  # Sorts alphabetically
#=> This also ensure reproducibility by having a known file order

#print(os.path.splitext(file_list[0]))
#print(h5_files)

h5_paths = []
for filename in h5_files:
    h5_paths.append(data_folder + filename) 

#print(len(h5_paths))

# Load anomaly filenames from a text file
with open("annotated_files.txt", "r") as f:
    anomaly_files = set(line.strip() for line in f if line.strip())
    
#Define anomaly paths before truncating h5_paths
anomaly_paths = [p for p in h5_paths if os.path.basename(p) in anomaly_files]

DEBUG_MODE = 0
if DEBUG_MODE:
    #WHILE DEBUGGING, ONLY USE A FEW FILES
    #NEED THIS FOR DEBUGGING - POTENTIALLY USES 13 GB OF MEMORY
    num_files = 200 # 1000 makes the kernel crash w 2400 MB memory
    h5_paths = h5_paths[:num_files]    

    #file_idx = 3567
    #h5_paths = h5_paths[file_idx-5 : file_idx+5 ]                                   

#full_dataset = ADCPDataset(h5_paths) # (data_dir)

normal_paths = [p for p in h5_paths if os.path.basename(p) not in anomaly_files]


# Example: 70% train, 20% val, 10% test
#total_size = len(full_dataset)
#train_size = int(0.7 * total_size)
#val_size = int(0.20 * total_size)
#test_size = total_size - train_size - val_size  # ensure all samples are used

#train_dataset, val_dataset, test_dataset = random_split(
#    full_dataset, [train_size, val_size, test_size]

#train_size = int(0.8 * len(full_dataset))
#val_size = len(full_dataset) - train_size
#train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

# Split anomaly files
an_train, an_temp = train_test_split(anomaly_paths, test_size=0.3, random_state=seed_num) #70/30 split
an_val, an_test = train_test_split(an_temp, test_size=0.33, random_state=seed_num) #20/10 split 

# Split normal files
n_train, n_temp = train_test_split(h5_paths, test_size=0.3, random_state=seed_num) #70/30 split
n_val, n_test = train_test_split(n_temp, test_size=0.33, random_state=seed_num) #20/10 split 

# Combine
train_files = an_train + n_train
val_files = an_val + n_val
test_files = an_test + n_test

#Create the datasets:
train_dataset = ADCPDataset(train_files)
val_dataset = ADCPDataset(val_files)
test_dataset = ADCPDataset(test_files)

print('train/val/test datasets grabbed')

#Set num_workers to 2x cpu cores (NO, get a warning with that), and pin_memory when using a GPU
train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True, num_workers=4, pin_memory=True)
#train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=config.batch_size, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=config.batch_size, shuffle=False, num_workers=4)


train/val/test datasets grabbed


/home/slonimer/torch_env25/lib/python3.11/site-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [4]:
#Example:
#if num_files is 20, then len(full_dataset) will be 60 (*3) because of the 3 beams
print(len(train_files))
print(len(val_files))
print(len(test_files))


2767
795
393


In [19]:
# Cell 4: Initialize Model and Loss - Resnet50
num_classes = 6 # full_dataset.num_classes
# model = TemporalCNN(input_channels=3, num_classes=num_classes)

variant = 'resnet50'   # or 'resnet18', 'resnet34', etc.
model = ResNetTemporalClassifier(
    num_classes=num_classes,
    pretrained=False,      # set False if you want to train from scratch
    variant=variant,     # options: 'resnet50', 'resnet101', 'resnet152' (but only 18,34,50 are avail pre-trained for now)
    resize=(224, 224)        # input size for ResNet
)
#In the "model" initialization above, 
#I've set the pre-trained to false, since no internet, but will load from pre-trained models I got on my VM


# --- Load pretrained weights manually (offline) ---
pretrained_path = f"/lustre10/scratch/slonimer/models/{variant}.pth"
state_dict = torch.load(pretrained_path, map_location='cpu')

# Filter out the fc layer weights (1000-class classifier)
filtered_state_dict = {k: v for k, v in state_dict.items() if not k.startswith("fc.")}

# only load matching keys (to ignore classifier layer mismatches)
missing, unexpected = model.backbone.load_state_dict(filtered_state_dict, strict=False)
print(f"✅ Loaded pretrained weights with {len(missing)} missing and {len(unexpected)} unexpected keys")


class_weights = get_class_weights(train_dataset, num_classes)                    # FIX THIS LATER -  UNCOMMENT THIS OR DEFINE MANUALLY BUT CORRECT WEIGHTS
print(class_weights)
#class_weights = torch.tensor([1.6761e-01, 3.8812e+01, 1.4015e+02, 9.8140e+02, 0.0000e+00, 0.0000e+00])

loss_fn = combined_loss(class_weights, alpha=config.loss_alpha)
optimizer = torch.optim.Adam(model.parameters(), lr=config.lr)


✅ Loaded pretrained weights with 2 missing and 0 unexpected keys
tensor([1.6761e-01, 3.8812e+01, 1.4015e+02, 9.8140e+02, 0.0000e+00, 0.0000e+00])


In [ ]:
# Cell 5: Train

best_model_path = f"best_model_{variant}.pt"
if USE_WANDB:
    model = train_model(model, train_loader, val_loader, optimizer, loss_fn, device, num_epochs=config.epochs, patience=5, USE_WANDB=USE_WANDB, best_model_path = best_model_path)
else:
    model, history = train_model(model, train_loader, val_loader, optimizer, loss_fn, device, num_epochs=config.epochs, patience=5, USE_WANDB=USE_WANDB, best_model_path = best_model_path)

# ([2, 3, 288, 102])
# => [batch, channels, time, range]
    
#Best result using ce and dice-loss:
#Epoch 7/20 | Train Loss: 0.7003 | Val Loss: 0.9850 | Val F1: 0.4931
#⏹️ Early stopping triggered.
    
# With ce and graduated dice-loss:
# Starting Validation on Epoch #  6
# Epoch 7/20 | Train Loss: 0.7778 | Val Loss: 0.9043 | Val F1: 0.4931
# #
# Starting Validation on Epoch #  7
# Epoch 8/20 | Train Loss: 0.7598 | Val Loss: 0.9866 | Val F1: 0.4931
# ⏹️ Early stopping triggered.



Starting Training on Epoch #  0
Loss (& total) on Batch #1: 1.519645094871521 (1.519645094871521)
Loss (& total) on Batch #2: 1.0346615314483643 (2.5543066263198853)
Loss (& total) on Batch #3: 0.5921858549118042 (3.1464924812316895)
Loss (& total) on Batch #4: 2.83632230758667 (5.982814788818359)
Loss (& total) on Batch #5: 0.576441764831543 (6.559256553649902)
Loss (& total) on Batch #6: 0.5786707401275635 (7.137927293777466)
Loss (& total) on Batch #7: 0.5579795837402344 (7.6959068775177)
Loss (& total) on Batch #8: 0.5194362998008728 (8.215343177318573)
Loss (& total) on Batch #9: 0.5143057107925415 (8.729648888111115)
Loss (& total) on Batch #10: 0.5095085501670837 (9.239157438278198)
Loss (& total) on Batch #11: 0.5061085224151611 (9.74526596069336)
Loss (& total) on Batch #12: 0.5171078443527222 (10.262373805046082)
Loss (& total) on Batch #13: 0.5030055642127991 (10.76537936925888)
Loss (& total) on Batch #14: 0.5043883919715881 (11.269767761230469)
Loss (& total) on Batch #15:

In [ ]:
#Profile memory from linux terminal with:
# $top -u slonimer

#8.2 g : Load data loaders
#1.4 g: More memory needed for running the model

#Notes: Running batch size of 16 for full data set takes a long time.  Should try doing larger batch size.
# No 2-power batch sizes are divisible by 3 (number of beams) but could try something larger, like 256


# Can also check GPUs:
# $ watch -n 1 nvidia-smi

# OR CPU utilization
# $ ps -u $USER -f | grep pt_data_worker


In [ ]:
#If the kernel dies part way through training,
# Load the most recent model and continue training

# Cell 4: Initialize Model and Loss
num_classes = 6 # full_dataset.num_classes

variant = 'resnet50'   # or 'resnet18', 'resnet34', etc.
model = ResNetTemporalClassifier(
    num_classes=num_classes,
    pretrained=False,      # set False if you want to train from scratch
    variant=variant,     # options: 'resnet50', 'resnet101', 'resnet152' (but only 18,34,50 are avail pre-trained for now)
    resize=(224, 224)        # input size for ResNet
)
#In the "model" initialization above, 
#I've set the pre-trained to false, since no internet, but will load from pre-trained models I got on my VM

# Load weights
model.load_state_dict(torch.load("best_model_resnet50_.pt", map_location='cuda'))
model = model.to('cuda')  # or 'cpu'

class_weights = get_class_weights(train_dataset, num_classes)                    # FIX THIS LATER -  UNCOMMENT THIS OR DEFINE MANUALLY BUT CORRECT WEIGHTS
print(class_weights)

loss_fn = combined_loss(class_weights, alpha=config.loss_alpha)
optimizer = torch.optim.Adam(model.parameters(), lr=config.lr)

# Cell 5: Train
best_model_path = f"best_model_{variant}.pt"
if USE_WANDB:
    model          = train_model(model, train_loader, val_loader, optimizer, loss_fn, device, num_epochs=config.epochs, patience=5, USE_WANDB=USE_WANDB, best_model_path = best_model_path)
else:
    model, history = train_model(model, train_loader, val_loader, optimizer, loss_fn, device, num_epochs=config.epochs, patience=5, USE_WANDB=USE_WANDB, best_model_path = best_model_path)


tensor([1.6761e-01, 3.8812e+01, 1.4015e+02, 9.8140e+02, 0.0000e+00, 0.0000e+00])
Starting Training on Epoch #  0
Loss (& total) on Batch #1: 0.5087339878082275 (0.5087339878082275)
Loss (& total) on Batch #2: 0.5012625455856323 (1.0099965333938599)
Loss (& total) on Batch #3: 0.5001527667045593 (1.5101493000984192)
Loss (& total) on Batch #4: 0.4999542236328125 (2.0101035237312317)
Loss (& total) on Batch #5: 0.4999334216117859 (2.5100369453430176)
Loss (& total) on Batch #6: 0.4999291002750397 (3.0099660456180573)
Loss (& total) on Batch #7: 0.4999285638332367 (3.509894609451294)
Loss (& total) on Batch #8: 0.49992835521698 (4.009822964668274)
Loss (& total) on Batch #9: 0.49992823600769043 (4.509751200675964)
Loss (& total) on Batch #10: 0.4999281167984009 (5.009679317474365)
Loss (& total) on Batch #11: 0.4999280571937561 (5.509607374668121)
Loss (& total) on Batch #12: 0.49992844462394714 (6.0095358192920685)
Loss (& total) on Batch #13: 0.49992841482162476 (6.509464234113693)
Loss

In [6]:
print(history)


{'train_loss': [0.9583881768968797, 0.8589790086111715, 0.8574826263131634, 0.8522834172171931, 0.8336483220900258, 0.8325665047572505, 0.82156113390961, 0.8176933988448112, 0.8018415827424296, 0.7963504906623594, 0.784831604890285, 0.7852150708917649, 0.7825517070389563, 0.7698381033635908, 0.7750539250912205, 0.7535960888189654, 0.751354037513656, 0.7524507615354753, 0.7423527644526574, 0.7395329321584394], 'val_loss': [0.8274070297678312, 0.792217128806644, 0.7939501139852736, 0.7924235314130783, 0.7700290646817949, 0.7498496580455039, 0.7518935592638122, 0.7520300754242473, 0.7267159074544907, 0.7344969949788518, 0.7293031530247794, 0.7422766718599532, 0.7212113978134261, 0.7469947520229552, 0.7144439145922661, 0.713933002617624, 0.7212710918651687, 0.7030177207456695, 0.7185809653666284, 0.7074187878105376], 'train_f1': [0.15546289138459493, 0.19007597932988193, 0.18941585088207896, 0.19484925846918627, 0.20334456890318764, 0.21256885120545368, 0.23521618564485366, 0.2263456753241

In [ ]:
#NEXT: Train on Resnet18

# Cell 4: Initialize Model and Loss
num_classes = 6 # full_dataset.num_classes

variant = 'resnet18'   # or 'resnet18', 'resnet34', etc.
model = ResNetTemporalClassifier(
    num_classes=num_classes,
    pretrained=False,      # set False if you want to train from scratch
    variant=variant,     # options: 'resnet50', 'resnet101', 'resnet152' (but only 18,34,50 are avail pre-trained for now)
    resize=(224, 224)        # input size for ResNet
)
#In the "model" initialization above, 
#I've set the pre-trained to false, since no internet, but will load from pre-trained models I got on my VM


# Load weights - Reload if crashes part way through
# model.load_state_dict(torch.load("best_model_resnet50_.pt", map_location='cuda'))
# model = model.to('cuda')  # or 'cpu'

# --- Load pretrained weights manually (offline) ---
pretrained_path = f"/lustre10/scratch/slonimer/models/{variant}.pth"
state_dict = torch.load(pretrained_path, map_location='cpu')
# Filter out the fc layer weights (1000-class classifier)
filtered_state_dict = {k: v for k, v in state_dict.items() if not k.startswith("fc.")}
# only load matching keys (to ignore classifier layer mismatches)
missing, unexpected = model.backbone.load_state_dict(filtered_state_dict, strict=False)
print(f"✅ Loaded pretrained weights with {len(missing)} missing and {len(unexpected)} unexpected keys")

class_weights = get_class_weights(train_dataset, num_classes)                    # FIX THIS LATER -  UNCOMMENT THIS OR DEFINE MANUALLY BUT CORRECT WEIGHTS
print(class_weights)

loss_fn = combined_loss(class_weights, alpha=config.loss_alpha)
optimizer = torch.optim.Adam(model.parameters(), lr=config.lr)

# Cell 5: Train

best_model_path = f"best_model_{variant}.pt"
if USE_WANDB:
    model          = train_model(model, train_loader, val_loader, optimizer, loss_fn, device, num_epochs=config.epochs, patience=5, USE_WANDB=USE_WANDB, best_model_path = best_model_path)
else:
    model, history = train_model(model, train_loader, val_loader, optimizer, loss_fn, device, num_epochs=config.epochs, patience=5, USE_WANDB=USE_WANDB, best_model_path = best_model_path)

In [ ]:
print(history)


In [24]:
# Cell 6: Load Best Model and Evaluate
model.load_state_dict(torch.load("best_model_20250508.pt"))
#model.load_state_dict(torch.load("best_model.pt"))
model.eval()

all_preds = []
all_labels = []

#for x, y in val_loader:
for x, y in test_loader:
    x = x.to(device)
    with torch.no_grad():
        out = model(x)
        #Need to reshape the outputs, and y to match dimensions:
        out = out.reshape(-1, out.shape[-1])  # (B*T, num_classes)
        #The prediction is the class with largest score per sample
        preds = torch.argmax(out, dim=1)

    #Need to reshape the outputs, and y to match dimensions:
    y = y.view(-1)                        # (B*T, )

    #Append the results
    all_preds.append(preds.cpu())
    all_labels.append(y)

y_pred = torch.cat(all_preds).numpy()
y_true = torch.cat(all_labels).numpy()

from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred))




/tmp/ipykernel_3219469/1187203441.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_model_20250508.pt"))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00       817
           1       0.98      0.98      0.98        47

    accuracy                           1.00       864
   macro avg       0.99      0.99      0.99       864
weighted avg       1.00      1.00      1.00       864



In [ ]:
'''

#Best model 2025-05-08

val_loader:
    
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    684575
           1       0.78      0.98      0.87      1913
           2       0.00      0.00      0.00       383
           3       0.44      0.89      0.59         9

    accuracy                           1.00    686880
   macro avg       0.56      0.72      0.62    686880
weighted avg       1.00      1.00      1.00    686880



test_loader:

           0       1.00      1.00      1.00    337877
           1       0.73      0.95      0.83      1257
           2       0.00      0.00      0.00       418
           3       0.00      0.00      0.00         0

    accuracy                           1.00    339552
   macro avg       0.43      0.49      0.46    339552
weighted avg       1.00      1.00      1.00    339552


'''

In [8]:
#Cell 7: Make test plots.  
#I want to push a file through the algorithm, and see how it performs

#IMPORT EVERYTHING:
import os
import torch
from torch.utils.data import DataLoader, random_split

from src.dataset_loader import ADCPDataset  # your custom dataset
from src.model import TemporalCNN # CNNClassifier  # your model
from src.utils import seed_everything, get_class_weights, combined_loss, train_model

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

'''
#DEBUG:
#I was having bugs in this, where it was saying it failed creating a primitive. This was found to solve the issue (but shouldn't be used when proper training/running)
import os
os.environ["TORCH_DISABLE_MKL"] = "1"  # optional: disables MKL
os.environ["ONEDNN_VERBOSE"] = "0"
os.environ["DNNL_VERBOSE"] = "0"
torch.backends.mkldnn.enabled = False
'''


# INITIALIZE THE MODEL
model_path = "/lustre10/scratch/slonimer/ML_ADCP/"
#model_path = "/scratch/slonimer/ML_ADCP/"
num_classes = 6 
#model = TemporalCNN(input_channels=3, num_classes=num_classes)
model = ResNetTemporalClassifier(
    num_classes=num_classes,
    pretrained=True,      # set False if you want to train from scratch
    variant='resnet50',     # options: 'resnet50', 'resnet101', 'resnet152'
    resize=(224, 224)        # input size for ResNet
)

# model.load_state_dict(torch.load(model_path + "best_model_20250505.pt" ))
model.load_state_dict(torch.load(model_path + "best_model_20250508.pt" ))
model.eval()


#Specify the data to use:
file_path = "/lustre10/scratch/slonimer/ML_ADCP/BACAX_24hr_h5/"
#file_path = "/scratch/slonimer/ML_ADCP/BACAX_24hr_h5/"
h5_filename = '20240406T000000_20240406T235959.h5'
#h5_filename = '20230701T000230_20230702T000229.h5'
h5_test_file = []
# h5_test_file.append(file_path + '20230701T000230_20230702T000229.h5') 
h5_test_file.append(file_path + h5_filename) 


#CLASSIFY THE TEST FILE
def classify_test_data(model, h5_test_file):
    test_file_dataset = ADCPDataset(h5_test_file)
    test_loader = DataLoader(test_file_dataset, batch_size=3, shuffle=False, num_workers=4)

    all_preds = []
    all_labels = []


    for x, y in test_loader:
        x = x.to(device)
        model = model.to(device)  # ← Add this line
        with torch.no_grad():
            out = model(x)
            #Need to reshape the outputs, and y to match dimensions:
            out = out.reshape(-1, out.shape[-1])  # (B*T, num_classes)
            #The prediction is the class with largest score per sample
            preds = torch.argmax(out, dim=1)

        #Need to reshape the outputs, and y to match dimensions:
        y = y.view(-1)                        # (B*T, )

        #Append the results
        all_preds.append(preds.cpu())
        all_labels.append(y)
        
    return x, y, preds


#Run the classification
x, y, preds = classify_test_data(model, h5_test_file)

/tmp/ipykernel_1775809/1511681119.py:31: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path + "best_model_20250508.pt" ))


In [25]:
from sklearn.metrics import classification_report
print(classification_report(y.cpu(), preds.cpu()))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       817
           1       0.98      0.98      0.98        47

    accuracy                           1.00       864
   macro avg       0.99      0.99      0.99       864
weighted avg       1.00      1.00      1.00       864



In [49]:
import h5py
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np


def get_segments(annotations, ann):
    if ann==1: #For annotations (may be multiclass)
        mask = np.diff(annotations) != 0 # Create a mask of where changes in annotations are non-zero
        #diffs = mask.astype(int)
    elif ann==0: #For predictions
        mask = annotations != 0 # Create a mask of where annotations are non-zero
        
    diffs = np.diff(mask.astype(int)) # Find the changes in the mask
    start_indices = np.where(diffs == 1)[0] + 1 # Start indices: where diff == 1 (0 → 1)
    end_indices = np.where(diffs == -1)[0] + 1 # End indices: where diff == -1 (1 → 0)
    # Handle edge cases: 
    if mask[0]: #if it starts with a non-zero 
        start_indices = np.r_[0, start_indices]

    if mask[-1]: # if it ends with a non-zero
        end_indices = np.r_[end_indices, len(annotations)]

    anomaly_segments = list(zip(start_indices, end_indices)) # Zip together
    return anomaly_segments


def plot_results(x, annotations, predictions, filename) :
    x = x.cpu()
    annotations = annotations.cpu()
    predictions = predictions.cpu()

    n_beams = x.shape[0]#[2]
    n_channels = x.shape[1]#[2]

    for beam in range(n_beams):
        fig, axs = plt.subplots(n_channels, 1, sharex=True, figsize=(12, 2.5*n_channels))
        if n_channels == 1:
            axs = [axs]

        #Get the annotations for this beam
        anomaly_segments = get_segments(annotations[beam].cpu().numpy(),ann = 1)
        pred_segments = get_segments(predictions[beam].cpu().numpy(), ann = 0)

        #print(anomaly_segments)
        #print(pred_segments)

        #Determine if any annotations present in this beam:
        cls_str = '' #Initialize as nothing
        ann = annotations[beam].cpu().numpy()
        if np.any(ann>0):
            cls = np.median(ann[ann>0])
            cls_str = ', class: {}'.format(int(cls))


        #Plot Velocity, backscatter, and correlation, for each beam
        for ch in range(n_channels):
            #Plot the Complex Data
            im = axs[ch].imshow(
                x[beam,ch,:,:].T, aspect='auto', origin='lower',
                #extent=[extent[0], extent[1], extent[2], extent[3]],
                #extent=[t_hours[0], t_hours[-1], 0, arrp.shape[1]-1],
                interpolation='nearest',
                cmap='jet',
            )

            #Set the figure title
            if beam == 0 and ch == 0:
                fig.suptitle('File: {}'.format(filename))
            
            #Set the subplot titles
            if ch == 0:
                axs[ch].set_title('Beam #{} {}'.format(beam+1, cls_str))   
           
            #Add labels and titles
            #axs[ch].set_ylabel("Range bin" if range_dim is not None else '')
            #axs[ch].set_title(f"{var} - Channel {ch+1}")

            #Add dashed vertical lines for annotations
            for start, end in anomaly_segments:
                if 0 <= start < x.shape[2]:
                    axs[ch].axvline(x=start, color='red', linestyle='dashed', alpha=0.7)
                if 0 <= end < x.shape[2]:
                    axs[ch].axvline(x=end, color='red', linestyle='dashed', alpha=0.7)

            #Add shading for predictions
            for start, end in pred_segments:
                if 0 <= start < x.shape[2] and 0 <= end <= x.shape[2]:
                    axs[ch].axvspan(start, end, color='black', alpha=0.3)

            #Add a colorbar
            fig.colorbar(im, ax=axs[ch], label='color')

        # -- Date formatting for X --
        axs[-1].xaxis_date()  # tells matplotlib to interpret x as dates
        axs[-1].xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
        fig.autofmt_xdate()  # Makes dates pretty (auto-rotates, etc.)

        #axs[-1].set_xlabel(time_dt[0].astype('datetime64[D]').astype(str))   # 'yyyy-mm-dd' date for xlabel
        #axs[-1].set_xlabel("Time (hours since start)")
        #fig.suptitle(f"{var} (shape={arr.shape})")
        plt.tight_layout()
        
        #if outdir:
        #    if not os.path.exists(outdir):
        #        os.makedirs(outdir)
        #    plt.savefig(f"{outdir}/{var}.png", dpi=120)
        #if show:
        #    plt.show()
        plt.show()
        #plt.close()

        
#

In [ ]:
#Get indices of start/end segments of anomalies
annotations = y.view(3,288)
predictions = preds.view(3,288)

plot_results(x, annotations, predictions, h5_filename)

In [ ]:
#For ALL anomalous files, plot the annotations and the prediction

# Load anomaly filenames from a text file
with open("annotated_files.txt", "r") as f:
    anomaly_files = set(line.strip() for line in f if line.strip())
    
#Define anomaly paths before truncating h5_paths
file_path = "/scratch/slonimer/ML_ADCP/BACAX_24hr_h5/"
anomaly_paths = []
for filename in anomaly_files:
    anomaly_paths.append(file_path + filename)
    
#    anomaly_paths = [p for p in h5_paths if os.path.basename(p) in anomaly_files]

#Run the classification
for anomaly_path in anomaly_paths:
    print(anomaly_path)
    #Predict the class
    x, y, preds = classify_test_data(model, [anomaly_path])
    
    #Get indices of start/end segments of anomalies
    annotations = y.view(3,288)
    predictions = preds.view(3,288)


    #Plot the results
    plot_results(x, annotations, predictions, os.path.basename(anomaly_path))

In [ ]:
#Look for False positives!

#Push all files through. If any are classified as drop-outs with more than 6 in a row (half an hour), make a plot

#Get a list of files from the directory:
data_folder = "/scratch/slonimer/ML_ADCP/BACAX_24hr_h5/"
#data_folder= r"F:\Documents\Projects\ML\ADCP_ML\h5_24h_files\\"
file_list = os.listdir(data_folder)
h5_files = {k for k in file_list if os.path.splitext(k)[1] == ".h5"}
#Files are inherently NOT in order in python! So if you want them in order, need to do this:
h5_files = sorted(h5_files)  # Sorts alphabetically

h5_paths = []
for filename in h5_files:
    h5_paths.append(data_folder + filename) 

'''
#For ALL anomalous files, plot the annotations and the prediction

# Load anomaly filenames from a text file
with open("annotated_files.txt", "r") as f:
    anomaly_files = set(line.strip() for line in f if line.strip())
    
#Define anomaly paths before truncating h5_paths
file_path = "/scratch/slonimer/ML_ADCP/BACAX_24hr_h5/"
anomaly_paths = []
for filename in anomaly_files:
    anomaly_paths.append(file_path + filename)
    
#    anomaly_paths = [p for p in h5_paths if os.path.basename(p) in anomaly_files]
'''

#Run the classification
for file_path in h5_paths:
    print(file_path)
    #Predict the class
    x, y, preds = classify_test_data(model, [file_path])
    
    #Get indices of start/end segments of anomalies
    annotations = y.view(3,288)
    predictions = preds.view(3,288)

    
    #Determine if any annotations present in any beam:
    do_plot = 0
    for beam in range(3):
        ann_beam = annotations[beam].cpu().numpy()
        pred_beam = predictions[beam].cpu().numpy()
        
        #If more than one hour predicted in a day in any beam:
        n_samples = 24 # 1 hour
        #n_samples = 12 # 1 hour
        if np.all(ann_beam==0) and np.sum(pred_beam>0)>n_samples:
            do_plot = 1
    
    if do_plot == 1:
        #Plot the results
        plot_results(x, annotations, predictions, os.path.basename(file_path))

    